```
--- Day 9: Sensor Boost ---

You've just said goodbye to the rebooted rover and left Mars when you receive a faint distress signal coming from the asteroid belt. It must be the Ceres monitoring station!

In order to lock on to the signal, you'll need to boost your sensors. The Elves send up the latest BOOST program - Basic Operation Of System Test.

While BOOST (your puzzle input) is capable of boosting your sensors, for tenuous safety reasons, it refuses to do so until the computer it runs on passes some checks to demonstrate it is a complete Intcode computer.

Your existing Intcode computer is missing one key feature: it needs support for parameters in relative mode.

Parameters in mode 2, relative mode, behave very similarly to parameters in position mode: the parameter is interpreted as a position. Like position mode, parameters in relative mode can be read from or written to.

The important difference is that relative mode parameters don't count from address 0. Instead, they count from a value called the relative base. The relative base starts at 0.

The address a relative mode parameter refers to is itself plus the current relative base. When the relative base is 0, relative mode parameters and position mode parameters with the same value refer to the same address.

For example, given a relative base of 50, a relative mode parameter of -7 refers to memory address 50 + -7 = 43.

The relative base is modified with the relative base offset instruction:

Opcode 9 adjusts the relative base by the value of its only parameter. The relative base increases (or decreases, if the value is negative) by the value of the parameter.
For example, if the relative base is 2000, then after the instruction 109,19, the relative base would be 2019. If the next instruction were 204,-34, then the value at address 1985 would be output.

Your Intcode computer will also need a few other capabilities:

The computer's available memory should be much larger than the initial program. Memory beyond the initial program starts with the value 0 and can be read or written like any other memory. (It is invalid to try to access memory at a negative address, though.)
The computer should have support for large numbers. Some instructions near the beginning of the BOOST program will verify this capability.
Here are some example programs that use these features:

109,1,204,-1,1001,100,1,100,1008,100,16,101,1006,101,0,99 takes no input and produces a copy of itself as output.
1102,34915192,34915192,7,4,7,99,0 should output a 16-digit number.
104,1125899906842624,99 should output the large number in the middle.
The BOOST program will ask for a single input; run it in test mode by providing it the value 1. It will perform a series of checks on each opcode, output any opcodes (and the associated parameter modes) that seem to be functioning incorrectly, and finally output a BOOST keycode.

Once your Intcode computer is fully functional, the BOOST program should report no malfunctioning opcodes when run in test mode; it should only output a single value, the BOOST keycode. What BOOST keycode does it produce?

Your puzzle answer was 3497884671.
```

In [101]:
pos(p) = p + 1;
ref(a,p) = a[pos(a[pos(p)])];
@enum Op plus=1 mul=2 stor=3 out=4 jnz=5 jez=6 lt=7 eq=8 off=9 halt=99
@enum Mode posi=0 imm=1 rel=2

struct Operation 
   op::Op 
   mode1::Mode
   mode2::Mode
   mode3::Mode
end

getOps(str::String) = getOps(parse(Int,str))
function getOps(num)  
    
        op = Op(num % 100)
        mode1 = Mode((num ÷ 100)  % 10)
        mode2 = Mode((num ÷ 1000)  % 10)
        mode3 = Mode((num ÷ 10000)  % 10)
        
        Operation(op,mode1,mode2,mode3)
end

function op1(operation,p,offset,prog) 
    println("$p: op1 $operation")
    if operation.mode1 == posi
        p1 = ref(prog,p+1)
    elseif operation.mode1 == rel
        error("implement")
    elseif operation.mode1 == imm
        p1 = prog[pos(p+1)]
    end  
    
    if operation.mode2 == posi
        p2 = ref(prog,p+2)
    elseif operation.mode2 == rel
        error("implement") 
    elseif operation.mode2 == imm
        p2 = prog[pos(p+2)]
    end  
 
    if operation.mode3 == posi        
        p3 = prog[pos(p+3)]
    elseif operation.mode3 == rel        
        error("implement")   
    elseif operation.mode3 == imm     
        p3 = prog[pos(p+3)]
    end   
    println("\tadding $p1 + $p2 and stroing at position [$p3]")    
    prog[pos(p3)] = p1 + p2
    p + 4
end

function op2(operation,p,offset,prog) 
    println("$p: op2 $operation")
    #println("entering op2 command in mode $operation.mode1")
    #println("offset: $offset")
    if operation.mode1 == posi
        p1 = ref(prog,p+1)
    elseif operation.mode1 == rel
        error("implement") 
    elseif operation.mode1 == imm
        p1 = prog[pos(p+1)]
    end  
    
    if operation.mode2 == posi
        p2 = ref(prog,p+2)
    elseif operation.mode2 == rel
        error("implement")        
    elseif operation.mode2 == imm
        p2 = prog[pos(p+2)]
    end  
    
    if operation.mode3 == posi        
        p3 = prog[pos(p+3)]
    elseif operation.mode3 == rel        
        error("implement")    
    elseif operation.mode3 == imm     
        p3 = prog[pos(p+3)]
    end
    println("\tmultiplying $p1 * $p2")
    prog[pos(p3)] = p1 * p2
    p + 4
end

function op3(operation,p,offset,prog,input) 
    println("$p: op3 $operation")
    println("\toffset: $offset, input: $input") 
    if operation.mode1 == posi
        p1 = prog[pos(p+1)]
        prog[pos(p1)] = input      
    elseif operation.mode1 == rel
        error("implement")  
        prog[pos(p1)] = input 
    elseif operation.mode1 == imm
        p1 = prog[pos(p+1)]
        prog[pos(p1)] = input 
    end     
    p + 2
end

function op4(operation,p,offset,prog,output) #out  
    println("$p: op4 $operation, output: $output")
    if operation.mode1 == posi
        p1 = ref(prog,p+1)
    elseif operation.mode1 == rel
        param = prog[pos(p+1)] # get the parameter
        off_p = param + offset # get the position with its offset
        p1 = prog[pos(off_p)] # read the memory address at offset + parameter
        #error("implement")  
    elseif operation.mode1 == imm
        p1 = prog[pos(p+1)]
    end  
    push!(output,p1)
    (p + 2,output)
end



# Opcode 5 is jump-if-true: 
# if the first parameter is non-zero, 
# it sets the instruction pointer to the value from the second parameter. 
# Otherwise, it does nothing.

function op5(operation,p,offset,prog) 
    println("$p: op5 $operation")   
    if operation.mode1 == posi
        p1 = ref(prog,p+1)
        println("\t(posi) checking value [$p1] stored in position $(p+1) to see if it non-zero")
    elseif operation.mode1 == rel
       error("implement")          
    elseif operation.mode1 == imm
        p1 = prog[pos(p+1)]
    end  
    
    if operation.mode2 == posi
        p2 = ref(prog,p+2)
    elseif operation.mode2 == rel
       error("implement")           
    elseif operation.mode2 == imm
        println("\t(imm) instruction pointer to jump to is stored at $(p+2)")
        p2 = prog[pos(p+2)]
    end  
    
    
    if p1 != 0
        println("\ttrue, so jumping to $p2")
        return p2
    else
        println("\tcontinuing to $(p+3)")
        return p + 3
    end
end

# Opcode 6 is jump-if-false: 
# if the first parameter is zero, 
# it sets the instruction pointer to the value from the second parameter. Otherwise, it does nothing.

function op6(operation,p,offset,prog) 
    println("$p: op6 $operation")      
    if operation.mode1 == posi
        p1 = ref(prog,p+1)
    elseif operation.mode1 == rel
       error("implement")                
    elseif operation.mode1 == imm
        p1 = prog[pos(p+1)]
    end  
    
    if operation.mode2 == posi
        p2 = ref(prog,p+2)
    elseif operation.mode2 == rel
       error("implement")                      
    elseif operation.mode2 == imm
        p2 = prog[pos(p+2)]
    end  
  
    if p1 == 0
        return p2
    else
        return p + 3
    end
end

# Opcode 7 is less than: 
# if the first parameter is less than the second parameter, 
# it stores 1 in the position given by the third parameter. Otherwise, it stores 0.

function op7(operation,p,offset,prog) 
    println("$p: op7 $operation")    
    if operation.mode1 == posi
        p1 = ref(prog,p+1)
    elseif operation.mode1 == rel
       error("implement")                     
    elseif operation.mode1 == imm
        p1 = prog[pos(p+1)]
    end  
    
    if operation.mode2 == posi
        p2 = ref(prog,p+2)
    elseif operation.mode2 == rel
       error("implement")         
    elseif operation.mode2 == imm
        p2 = prog[pos(p+2)]
    end  
    
    result = 0
    if p1 < p2
        result = 1
    end
    
    if operation.mode3 == posi       
        p3 = prog[pos(p+3)]
        #p3 = ref(prog,p+1)
    elseif operation.mode3 == rel        
       error("implement")   
    elseif operation.mode3 == imm        
        p3 = prog[pos(p+3)]
    end
    println("\tabout to compare $p1 < $p2 to load [$result] into position [$(p3)]")    
    
    prog[pos(p3)] = result

    p + 4
end

# Opcode 8 is equals: 
# if the first parameter is equal to the second parameter, 
# it stores 1 in the position given by the third parameter. Otherwise, it stores 0.

function op8(operation,p,offset,prog) 
    println("$p: op8 $operation")   
    if operation.mode1 == posi
        p1 = ref(prog,p+1)
    elseif operation.mode1 == rel
       error("implement")
    elseif operation.mode1 == imm
        p1 = prog[pos(p+1)]
    end  
    
    if operation.mode2 == posi         
        p2 = ref(prog,p+2)
    elseif operation.mode2 == rel         
       error("implement")
    elseif operation.mode2 == imm      
        p2 = prog[pos(p+2)]
    end  
    
    result = 0
    if p1 == p2
        result = 1
    end
        
    if operation.mode3 == posi        
        p3 = prog[pos(p+3)]
    elseif operation.mode3 == rel     
       error("implement")   
    elseif operation.mode3 == imm        
        p3 = prog[pos(p+3)]
    end    
    println("\tabout to compare $p1 and $p2 to load into $(p3)")
    prog[pos(p3)] = result

    p + 4
end

#Opcode 9 adjusts the relative base by the value of its only parameter. 
#The relative base increases  
#    (or decreases, if the value is negative) 
#     by the value of the parameter.
#For example, if the relative base is 2000, 
#    then after the instruction 109,19, 
#    the relative base would be 2019. 
#If the next instruction were 204,-34, 
#    then the value at address 1985 would be output.

function op9(operation,p,offset,prog) 
    println("$p: op9 $operation")    
    #println("entering offset command in mode $operation.mode1")
    #println("offset: $offset")
    if operation.mode1 == posi
        p1 = ref(prog,p+1)
    elseif operation.mode1 == rel
        param = prog[pos(p+1)]
        p1 = prog[pos(offset+param)]       
    elseif operation.mode1 == imm
        p1 = prog[pos(p+1)]
    end  
    #println("adjusting offset command by $p1")
    offset = offset + p1
    #println("offset: $offset")
    (p + 2,offset)
end





function execute(op::Operation, p, offset, prog, input,output)
    #println("offset: $offset")
    if op.op == plus
        return (op1(op,p,offset,prog),input,output,offset,false)
    end
    if op.op == mul
        return (op2(op,p,offset,prog),input,output,offset,false)
    end
    if op.op == stor
        if length(input) == 0 
            return (p,input,output,true)
        end
        next = input[1]
        input=input[2:end]
        return (op3(op,p,offset,prog,next),input,output,offset,false)
    end
    if op.op == out
        (p1,output) = op4(op,p,offset,prog,output)
        #println(output)
        return (p1,input,output,offset,false)
    end
    if op.op == jnz
        return (op5(op,p,offset,prog),input,output,offset,false)
    end
    if op.op == jez
        return (op6(op,p,offset,prog),input,output,offset,false)
    end   
    if op.op == lt
        return (op7(op,p,offset,prog),input,output,offset,false)
    end   
    if op.op == eq
        return (op8(op,p,offset,prog),input,output,offset,false)
    end   
    if op.op == off
        (p,offset) = op9(op,p,offset,prog)
        return (p,input,output,offset,false)
    end 
    
    error("panic executing unknown op")
end

function execute(prog,p,offset,input,output)
    push!(prog,zeros(Int,100000)...)
    op = getOps(prog[pos(p)])
    while op.op != halt
        (p,input,output,offset,halt) = execute(op,p,offset,prog,input,output)
        if halt
            return (prog,p,output,:pause)
        end
        op = getOps(prog[pos(p)])
    end 
    (prog,p,output,:over)
end

function execute(prog)
  execute(prog,0,0,[],[])
end

(prog,p,output,state) = execute([1,9,10,3,2,3,11,0,99,30,40,50]) # == [3500,9,10,70,2,3,11,0,99,30,40,50])
println(prog[1:12])
#println(execute([1,0,0,0,99]) == [2,0,0,0,99])
#println(execute([2,3,0,3,99]) == [2,3,0,6,99])
#println(execute([2,4,4,5,99,0]) == [2,4,4,5,99,9801])
#println(execute([1,1,1,4,99,5,6,0,99]) == [ 30,1,1,4,2,5,6,0,99])

# The above example program uses an input instruction to ask for a single number. 
# The program will then 
# output 999 if the input value is below 8,
# output 1000 if the input value is equal to 8, 
# or output 1001 if the input value is greater than 8.

(prog,p,output,state) =  (execute(
    [3,21,1008,21,8,20,1005,20,22,107,8,21,20,1006,20,31,
1106,0,36,98,0,0,1002,21,125,20,4,20,1105,1,46,104,
999,1105,1,46,1101,1000,1,20,4,20,1105,1,46,98,99]
    ,0,0,[-8],[]))
println("\n\ntest $output\n\n")
(prog,p,output,state) =  (execute(
    [3,21,1008,21,8,20,1005,20,22,107,8,21,20,1006,20,31,
1106,0,36,98,0,0,1002,21,125,20,4,20,1105,1,46,104,
999,1105,1,46,1101,1000,1,20,4,20,1105,1,46,98,99]
    ,0,0,[8],[]))
println("\n\ntest $output\n\n")
(prog,p,output,state) =  execute(
    [3,21,1008,21,8,20,1005,20,22,107,8,21,20,1006,20,31,
1106,0,36,98,0,0,1002,21,125,20,4,20,1105,1,46,104,
999,1105,1,46,1101,1000,1,20,4,20,1105,1,46,98,99]
    ,0,0,[16],[]);

println("\n\ntest $output\n\n")

0: op1 Operation(plus, posi, posi, posi)
	adding 30 + 40 and stroing at position [3]
4: op2 Operation(mul, posi, posi, posi)
	multiplying 70 * 50
[3500, 9, 10, 70, 2, 3, 11, 0, 99, 30, 40, 50]
0: op3 Operation(stor, posi, posi, posi)
	offset: 0, input: -8
2: op8 Operation(eq, posi, imm, posi)
	about to compare -8 and 8 to load into 20
6: op5 Operation(jnz, posi, imm, posi)
	(posi) checking value [0] stored in position 7 to see if it non-zero
	(imm) instruction pointer to jump to is stored at 8
	continuing to 9
9: op7 Operation(lt, imm, posi, posi)
	about to compare 8 < -8 to load [0] into position [20]
13: op6 Operation(jez, posi, imm, posi)
31: op4 Operation(out, imm, posi, posi), output: Any[]
33: op5 Operation(jnz, imm, imm, posi)
	(imm) instruction pointer to jump to is stored at 35
	true, so jumping to 46


test Any[999]


0: op3 Operation(stor, posi, posi, posi)
	offset: 0, input: 8
2: op8 Operation(eq, posi, imm, posi)
	about to compare 8 and 8 to load into 20
6: op5 Operation(j

```109,1,204,-1,1001,100,1,100,1008,100,16,101,1006,101,0,99 takes no input and produces a copy of itself as output.```

In [102]:
(prog,p,output,state) = execute([109,1,204,-1,1001,100,1,100,1008,100,16,101,1006,101,0,99],0,0,[],[])
(prog[1:16],output)

0: op9 Operation(off, imm, posi, posi)
2: op4 Operation(out, rel, posi, posi), output: Any[]
4: op1 Operation(plus, posi, imm, posi)
	adding 0 + 1 and stroing at position [100]
8: op8 Operation(eq, posi, imm, posi)
	about to compare 1 and 16 to load into 101
12: op6 Operation(jez, posi, imm, posi)
0: op9 Operation(off, imm, posi, posi)
2: op4 Operation(out, rel, posi, posi), output: Any[109]
4: op1 Operation(plus, posi, imm, posi)
	adding 1 + 1 and stroing at position [100]
8: op8 Operation(eq, posi, imm, posi)
	about to compare 2 and 16 to load into 101
12: op6 Operation(jez, posi, imm, posi)
0: op9 Operation(off, imm, posi, posi)
2: op4 Operation(out, rel, posi, posi), output: Any[109, 1]
4: op1 Operation(plus, posi, imm, posi)
	adding 2 + 1 and stroing at position [100]
8: op8 Operation(eq, posi, imm, posi)
	about to compare 3 and 16 to load into 101
12: op6 Operation(jez, posi, imm, posi)
0: op9 Operation(off, imm, posi, posi)
2: op4 Operation(out, rel, posi, posi), output: Any[109

([109, 1, 204, -1, 1001, 100, 1, 100, 1008, 100, 16, 101, 1006, 101, 0, 99], Any[109, 1, 204, -1, 1001, 100, 1, 100, 1008, 100, 16, 101, 1006, 101, 0, 99])

In [103]:
(prog,p,output,state) = execute([109,1,4,0,1001,100,1,100,1008,100,16,101,1006,101,0,99],0,0,[],[])
(prog[1:16],output)

0: op9 Operation(off, imm, posi, posi)
2: op4 Operation(out, posi, posi, posi), output: Any[]
4: op1 Operation(plus, posi, imm, posi)
	adding 0 + 1 and stroing at position [100]
8: op8 Operation(eq, posi, imm, posi)
	about to compare 1 and 16 to load into 101
12: op6 Operation(jez, posi, imm, posi)
0: op9 Operation(off, imm, posi, posi)
2: op4 Operation(out, posi, posi, posi), output: Any[109]
4: op1 Operation(plus, posi, imm, posi)
	adding 1 + 1 and stroing at position [100]
8: op8 Operation(eq, posi, imm, posi)
	about to compare 2 and 16 to load into 101
12: op6 Operation(jez, posi, imm, posi)
0: op9 Operation(off, imm, posi, posi)
2: op4 Operation(out, posi, posi, posi), output: Any[109, 109]
4: op1 Operation(plus, posi, imm, posi)
	adding 2 + 1 and stroing at position [100]
8: op8 Operation(eq, posi, imm, posi)
	about to compare 3 and 16 to load into 101
12: op6 Operation(jez, posi, imm, posi)
0: op9 Operation(off, imm, posi, posi)
2: op4 Operation(out, posi, posi, posi), output: A

([109, 1, 4, 0, 1001, 100, 1, 100, 1008, 100, 16, 101, 1006, 101, 0, 99], Any[109, 109, 109, 109, 109, 109, 109, 109, 109, 109, 109, 109, 109, 109, 109, 109])

In [104]:
#1102,34915192,34915192,7,4,7,99,0 should output a 16-digit number.
(prog,p,output,state) = execute([1102,34915192,34915192,7,4,7,99,0],0,0,[],[])
length(string(output[1]))

0: op2 Operation(mul, imm, imm, posi)
	multiplying 34915192 * 34915192
4: op4 Operation(out, posi, posi, posi), output: Any[]


16

In [105]:
#104,1125899906842624,99 should output the large number in the middle.
(prog,p,output,state) = execute([104,1125899906842624,99],0,0,[],[])
string(output[1])

0: op4 Operation(out, imm, posi, posi), output: Any[]


"1125899906842624"

In [106]:
#Once your Intcode computer is fully functional, 
#the BOOST program should report no malfunctioning opcodes when run in test mode; 
#it should only output a single value, the BOOST keycode. What BOOST keycode does it produce?
#The BOOST program will ask for a single input; run it in test mode by providing it the value 1
# code shoule be 3497884671
using DelimitedFiles
prog = readdlm("input.txt", '\t', Int, ',')
(prog,p,output,state) = execute(prog[:,1],0,0,[1],[])
output

0: op2 Operation(mul, imm, imm, posi)
	multiplying 34463338 * 34463338
4: op7 Operation(lt, posi, imm, posi)
	about to compare 1187721666102244 < 34463338 to load [0] into position [63]
8: op5 Operation(jnz, posi, imm, posi)
	(posi) checking value [0] stored in position 9 to see if it non-zero
	(imm) instruction pointer to jump to is stored at 10
	continuing to 11
11: op1 Operation(plus, imm, imm, posi)
	adding 0 + 3 and stroing at position [1000]
15: op9 Operation(off, imm, posi, posi)
17: op9 Operation(off, rel, posi, posi)
19: op9 Operation(off, posi, posi, posi)
21: op9 Operation(off, rel, posi, posi)
23: op9 Operation(off, rel, posi, posi)
25: op3 Operation(stor, rel, posi, posi)
	offset: 1000, input: 1


LoadError: implement